**ЧАСТЬ 1**

*Шаш 1 - импорт всего необходимого*

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from pymystem3 import Mystem
mystem = Mystem()
import spacy
import flair
import nltk
from string import punctuation
from sklearn.metrics import accuracy_score
import re
from flair.models import SequenceTagger

*Шаг 2 - разметка текстов для русского и английского*

In [ ]:
russian_text = ['До меня доносится легкий запах духов',
'Русичка запретила болтать на уроках',
'Была бонбандировка города',
'Сегодня у нас была ген. уборка',
'Махабхарата ужасный сериал',
'Печь печенья всегда весело',
'Многочасовой рейс из Москвы в Петербург',
'Я наелась до отвала',
'Я смеялась без удержу',
'Я учусь в вышке',
'Надеюсь она приедет',
'Она мне сказала шо мазик вредный для здоровья',
'Вчера изучала со студентами свежий проект закона о Госсовете',
'С пылу с жару только из печки пирожок', 
'Подробности и перспективы пустоты разъяснила народной газете Московский комсомолец',
'Ща буду есть бутир с тофу и мазиком',
'Выход на сушу есть величайшая геополитическая катастрофа нашего миллиардолетия',
'Твиттер моя самоя любимая соцсеть',
'Хотя фейсбук тоже ничего',
'А вконтакте полный ужас']

russian_tag = ['PREP PRON VERB ADJ NOUN NOUN',
                'NOUN VERB VERB PREP NOUN',
                'VERB NOUN NOUN',
                'ADV PREP PRON VERB ADJ NOUN',
                'NOUN ADJ NOUN',
                'VERB NOUN ADV ADV',
                'ADJ NOUN PREP NOUN PREP NOUN',
                'PRON VERB PREP NOUN',
                'PRON VERB PREP NOUN',
                'PRON VERB PREP NOUN',
               'VERB PRON VERB',
               'PRON PRON VERB CONJ NOUN ADJ PREP NOUN',
                'ADV VERB PREP NOUN ADJ NOUN NOUN PREP NOUN',
                'PREP NOUN PREP NOUN ADV PREP NOUN NOUN',
                'NOUN CONJ NOUN NOUN VERB ADJ NOUN ADJ NOUN',
                'ADV VERB VERB NOUN PREP NOUN CONJ NOUN',
                'NOUN PREP NOUN VERB ADJ ADJ NOUN PRON NOUN',
                'NOUN PRON ADJ ADJ NOUN',
                'CONJ NOUN ADV NOUN',
                'CONJ NOUN ADJ NOUN']



In [ ]:
english_text = ['i goed to the shop',
              'twitter is my absolutely fav social media',
              'facebook is the way to baaad',
              'I totally love your twts',
              'This is a subtweet of anyone in particular and it is going to annoy some people',
             'I have a lot of PhD followers and friends and I am a stubborn guy myself',
             'but one thing I am noticed is when someone gets a PhD it become 10 harder convince them they are wrong',
              'I have actually found the opposite in my discipline',
             'The PhD are generally open to critique and questioning of their work',
              'It is the people who have an MBA but no practical business experience that are absolute monsters'

              ]

english_tag = ['PRON VERB PART ART NOUN',
               'NOUN VERB PRON ADV ADJ ADJ NOUN',
               'NOUN VERB ART NOUN PART ADJ',
               'PRON ADV VERB PRON NOUN',
               'DET VERB ART NOUN PREP PRON PREP ADJ CONJ PRON VERB VERB PART VERB ADJ NOUN',
               'PRON VERB ART ADV PREP NOUN NOUN CONJ NOUN CONJ PRON VERB ART ADJ NOUN PRON',
               'CONJ NUM NOUN PRON VERB VERB VERB ADV PRON VERB ART NOUN PRON VERB NUM ADJ VERB PRON PRON VERB ADJ',
               'PRON VERB ADV VERB ART ADJ PREP PRON NOUN',
               'ART NOUN VERB ADV ADJ PART NOUN CONJ NOUN PREP PRON NOUN',
               'PRON VERB ART NOUN ADV VERB ART NOUN CONJ PART ADJ ADJ NOUN DET VERB ADJ NOUN'
               ]
               

**Почему я выбрала именно эти тексты?**

В русском много словоформ, которые могут быть неизвестны парсеру - шо, мазик и тд. Также есть сокращения - ген. уборка, которые часто определяются неверно (сужу по опыту с TurkuNLP). Также есть опечатки типа "бонбардировка"

В английском также есть вероятно неизвестные парсеру словоформы (например, неправильное образование прошедшего времени от глагола do - doed). Также есть много сокращений типа PHD, которые, как правило, тоже не очень хорошо распознаются парсерами 

*Шаг 3 - очистка текста*

In [ ]:
def clean_my_texts(texts, tags):
  clean_texts = []
  new_tags = []
  for i in texts:
    i = i.lower()
    for znak in punctuation:
      if znak in i:
        i = i.replace(znak, '')
        i = i.replace('  ', ' ')
    i = i.split(' ')
    for n in i:
      clean_texts.append(n)
    new_tags = []
  for tag in tags:
    tag = tag.split(' ')
    for t in tag:
      new_tags.append(t)
  return clean_texts, new_tags

In [ ]:
russian_clean = clean_my_texts(russian_text, russian_tag)
print(russian_clean)

(['до', 'меня', 'доносится', 'легкий', 'запах', 'духов', 'русичка', 'запретила', 'болтать', 'на', 'уроках', 'была', 'бонбандировка', 'города', 'сегодня', 'у', 'нас', 'была', 'ген', 'уборка', 'махабхарата', 'ужасный', 'сериал', 'печь', 'печенья', 'всегда', 'весело', 'многочасовой', 'рейс', 'из', 'москвы', 'в', 'петербург', 'я', 'наелась', 'до', 'отвала', 'я', 'смеялась', 'без', 'удержу', 'я', 'учусь', 'в', 'вышке', 'надеюсь', 'она', 'приедет', 'она', 'мне', 'сказала', 'шо', 'мазик', 'вредный', 'для', 'здоровья', 'вчера', 'изучала', 'со', 'студентами', 'свежий', 'проект', 'закона', 'о', 'госсовете', 'с', 'пылу', 'с', 'жару', 'только', 'из', 'печки', 'пирожок', 'подробности', 'и', 'перспективы', 'пустоты', 'разъяснила', 'народной', 'газете', 'московский', 'комсомолец', 'ща', 'буду', 'есть', 'бутир', 'с', 'тофу', 'и', 'мазиком', 'выход', 'на', 'сушу', 'есть', 'величайшая', 'геополитическая', 'катастрофа', 'нашего', 'миллиардолетия', 'твиттер', 'моя', 'самоя', 'любимая', 'соцсеть', 'хотя', 

In [ ]:
english_clean = clean_my_texts(english_text, english_tag)
print(english_clean)

(['i', 'goed', 'to', 'the', 'shop', 'twitter', 'is', 'my', 'absolutely', 'fav', 'social', 'media', 'facebook', 'is', 'the', 'way', 'to', 'baaad', 'i', 'totally', 'love', 'your', 'twts', 'this', 'is', 'a', 'subtweet', 'of', 'anyone', 'in', 'particular', 'and', 'it', 'is', 'going', 'to', 'annoy', 'some', 'people', 'i', 'have', 'a', 'lot', 'of', 'phd', 'followers', 'and', 'friends', 'and', 'i', 'am', 'a', 'stubborn', 'guy', 'myself', 'but', 'one', 'thing', 'i', 'am', 'noticed', 'is', 'when', 'someone', 'gets', 'a', 'phd', 'it', 'become', '10', 'harder', 'convince', 'them', 'they', 'are', 'wrong', 'i', 'have', 'actually', 'found', 'the', 'opposite', 'in', 'my', 'discipline', 'the', 'phd', 'are', 'generally', 'open', 'to', 'critique', 'and', 'questioning', 'of', 'their', 'work', 'it', 'is', 'the', 'people', 'who', 'have', 'an', 'mba', 'but', 'no', 'practical', 'business', 'experience', 'that', 'are', 'absolute', 'monsters'], ['PRON', 'VERB', 'PART', 'ART', 'NOUN', 'NOUN', 'VERB', 'PRON', 'A

**ЧАСТЬ 2 - тестируем все парсеры**

**PYMORPHY**

In [ ]:
pymorphy_tags = []
pymorphy_convert = []
m = MorphAnalyzer()

for i in russian_clean[0]:
  parse = m.parse(i)[0].tag.POS
  pymorphy_tags.append(parse)

for tag in pymorphy_tags:  # меняем теги на нужные нам

  tag = re.sub('NUMR$', 'NUM', tag)
  tag = re.sub('ADVB$', 'ADV', tag)
  tag = re.sub('COMP$', 'ADJ', tag)
  tag = re.sub('GRND$', 'VERB', tag)
  tag = re.sub('PRCL$', 'PART', tag)
  tag = re.sub('ADJF$', 'ADJ', tag)
  tag = re.sub('ADJS$', 'ADJ', tag)
  tag = re.sub('INFN$', 'VERB', tag)
  tag = re.sub('PRTF$', 'VERB', tag)
  tag = re.sub('PRED$', 'ADV', tag)
  tag = re.sub('PRTS$', 'VERB', tag)
  tag = re.sub('NPRO$', 'PRON', tag)
  
  pymorphy_convert.append(tag)

print(pymorphy_convert)

['PREP', 'PRON', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB', 'PREP', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'ADV', 'PREP', 'PRON', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'ADV', 'ADV', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PRON', 'VERB', 'PREP', 'NOUN', 'PRON', 'VERB', 'PREP', 'VERB', 'PRON', 'VERB', 'PREP', 'NOUN', 'VERB', 'PRON', 'VERB', 'PRON', 'PRON', 'VERB', 'PRON', 'NOUN', 'ADJ', 'PREP', 'NOUN', 'ADV', 'VERB', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'ADV', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'ADV', 'VERB', 'VERB', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'PART', 'ADV', 'CONJ', 'NOUN', 'ADJ', 'NOUN']


In [ ]:
accuracy_score(russian_clean[1], pymorphy_convert)

0.9196428571428571

**NATASHA**

In [ ]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [ ]:
natasha = []
n = ''
for i in russian_clean[0]:
  
  n += i + ' '
doc = Doc(n)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for d in doc.tokens:

  natasha.append(d.pos)
natasha_convert = []

for nat in natasha:  # меняем теги на нужные нам

  nat = re.sub('AUX$', 'VERB', nat)
  nat = re.sub('ADP$', 'PREP', nat)
  nat = re.sub('CCONJ$', 'CONJ', nat)
  nat = re.sub('SCONJ$', 'CONJ', nat)
  natasha_convert.append(nat)

print(natasha_convert)

['PREP', 'PRON', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB', 'PREP', 'NOUN', 'VERB', 'VERB', 'NOUN', 'ADV', 'PREP', 'PRON', 'VERB', 'NOUN', 'ADJ', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'ADV', 'ADV', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PRON', 'VERB', 'PREP', 'NOUN', 'PRON', 'VERB', 'PREP', 'NOUN', 'PRON', 'VERB', 'PREP', 'NOUN', 'PRON', 'PRON', 'VERB', 'PRON', 'PRON', 'VERB', 'PART', 'ADV', 'ADJ', 'PREP', 'NOUN', 'ADV', 'VERB', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PART', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'ADV', 'VERB', 'VERB', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'DET', 'NOUN', 'NOUN', 'DET', 'ADJ', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'ADV', 'PRON', 'CONJ', 'ADV', 'ADJ', 'NOUN']


In [ ]:
accuracy_score(russian_clean[1], natasha_convert)

0.875

**MYSTEM**

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz  # иначе pipe крашится и вообще что-то страшное происходит
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-18 15:52:30--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskmar13.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-18 15:52:30--  http://cache-mskmar13.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskmar13.cdn.yandex.net (cache-mskmar13.cdn.yandex.net)... 5.45.222.86, 2a02:6b8:0:2b03::86
Connecting to cache-mskmar13.cdn.yandex.net (cache-mskmar13.cdn.yandex.net)|5.45.222.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[================

In [ ]:
mystem_tags = []
mystem_convert = []

for i in russian_clean[0]:
  m = mystem.analyze(i)
  try:
    p = (m[0]['analysis'][0]['gr']).split('=')[0].split(',')[0]
  except:
    IndexError
  mystem_tags.append(p)

for tag in mystem_tags:  # меняем теги на нужные нам

  tag = re.sub('A$', 'ADJ', tag)
  tag = re.sub('SPRO$', 'PRON', tag)
  tag = re.sub('PR$', 'PREP', tag)
  tag = re.sub('ADVPRO$', 'ADV', tag)
  tag = re.sub('ANUM$', 'NUM', tag)
  tag = re.sub('V$', 'VERB', tag)
  tag = re.sub('APRO$', 'PRON', tag)
  tag = re.sub('S$', 'NOUN', tag)
  mystem_convert.append(tag)

print(mystem_convert)

['PREP', 'PRON', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB', 'PREP', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'ADVERB', 'PREP', 'PRON', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'ADVERB', 'ADVERB', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'PRON', 'VERB', 'PREP', 'NOUN', 'PRON', 'VERB', 'PREP', 'VERB', 'PRON', 'VERB', 'NOUN', 'NOUN', 'VERB', 'PRON', 'VERB', 'PRON', 'PRON', 'VERB', 'VERB', 'NOUN', 'ADJ', 'PREP', 'NOUN', 'ADVERB', 'VERB', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'INTJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PART', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'VERB', 'ADVERB', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'PRON', 'NOUN', 'NOUN', 'PRON', 'ADVERB', 'ADJ', 'NOUN', 'PART', 'NOUN', 'PART', 'ADVERB', 'PART', 'NOUN', 'ADJ', 'NOUN']


In [ ]:
accuracy_score(russian_clean[1], mystem_convert)

0.8035714285714286

**SPACY**

In [ ]:
tags_spacy = []
spacy_convert = []
nlp = spacy.load("en_core_web_sm")
s = ''
for word in english_clean[0]:  # конвертируем в строку 
  s = s + word + ' '
doc = nlp(s)

for token in doc:
    tags_spacy.append(token.pos_)

for tag in tags_spacy:  # заменяем теги на нужные нам

  tag = re.sub('PROPN$', 'NOUN', tag)
  tag = re.sub('ADP$', 'PREP', tag)
  tag = re.sub('CCONJ$', 'CONJ', tag)
  tag = re.sub('AUX$', 'VERB', tag)
  tag = re.sub('SCONJ$', 'CONJ', tag)

  spacy_convert.append(tag)
  
print(spacy_convert)

['PRON', 'VERB', 'PREP', 'DET', 'NOUN', 'NOUN', 'VERB', 'DET', 'ADV', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PREP', 'NOUN', 'PRON', 'ADV', 'VERB', 'DET', 'NOUN', 'DET', 'VERB', 'DET', 'NOUN', 'PREP', 'PRON', 'PREP', 'ADJ', 'CONJ', 'PRON', 'VERB', 'VERB', 'PART', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'CONJ', 'PRON', 'VERB', 'DET', 'ADJ', 'NOUN', 'PRON', 'CONJ', 'NUM', 'NOUN', 'PRON', 'VERB', 'VERB', 'VERB', 'ADV', 'PRON', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'NUM', 'ADJ', 'PREP', 'PRON', 'PRON', 'VERB', 'ADJ', 'PRON', 'VERB', 'ADV', 'VERB', 'DET', 'ADJ', 'PREP', 'DET', 'NOUN', 'DET', 'NOUN', 'VERB', 'ADV', 'ADJ', 'PREP', 'NOUN', 'CONJ', 'VERB', 'PREP', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'CONJ', 'DET', 'ADJ', 'NOUN', 'NOUN', 'DET', 'VERB', 'ADJ', 'NOUN']


In [ ]:
accuracy_score(english_clean[1], spacy_convert)

0.7807017543859649

**NLTK**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk_convert = []
s = ''
for word in english_clean[0]:  # конвертируем в строку
  s = s + word + ' '
tokenize = nltk.pos_tag(nltk.word_tokenize(s))
#print(tokenize)

for tok in tokenize:  # заменяем теги на нужные нам

  if tok[1] == 'DT':
    nltk_convert.append('DET')
  elif tok[1] == 'IN':
    nltk_convert.append('PREP')
  elif tok[1] == 'CC':
    nltk_convert.append('CONJ')
  elif tok[1] == 'TO':
    nltk_convert.append('PART')
  elif tok[1] == 'NN' or tok[1] == 'NNS' or tok[1] == 'NNPS' or tok[1] == 'NNP':
    nltk_convert.append('NOUN')
  elif tok[1] == 'PRP' or tok[1] == 'PRP$':
    nltk_convert.append('PRO')
  elif tok[1] == 'RB' or tok[1] == 'WRB' or tok[1] == 'WP' or tok[1] == 'WDT':
    nltk_convert.append('ADV')
  elif tok[1] == 'CD':
    nltk_convert.append('NUM')
  elif tok[1] == 'UH':
    nltk_convert.append('ADJ')
  elif tok[1] == 'VB' or tok[1] == 'VBD' or tok[1] == 'VBN' or tok[1] == 'VBP' or tok[1] == 'VBG' or tok[1] == 'VBZ':
    nltk_convert.append('VERB')
  elif tok[1] == 'JJ' or tok[1] == 'JJR' or tok[1] == 'JJS':
    nltk_convert.append('ADJ')
  else:
    nltk_convert.append(tok[1])
    
print(nltk_convert)

['NOUN', 'NOUN', 'PART', 'DET', 'NOUN', 'NOUN', 'VERB', 'PRO', 'ADJ', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PART', 'VERB', 'ADJ', 'ADV', 'VERB', 'PRO', 'NOUN', 'DET', 'VERB', 'DET', 'NOUN', 'PREP', 'NOUN', 'PREP', 'ADJ', 'CONJ', 'PRO', 'VERB', 'VERB', 'PART', 'VERB', 'DET', 'NOUN', 'VERB', 'VERB', 'DET', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'CONJ', 'VERB', 'VERB', 'DET', 'ADJ', 'NOUN', 'PRO', 'CONJ', 'NUM', 'NOUN', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADV', 'NOUN', 'VERB', 'DET', 'NOUN', 'PRO', 'VERB', 'NUM', 'ADJ', 'NOUN', 'PRO', 'PRO', 'VERB', 'ADJ', 'NOUN', 'VERB', 'ADV', 'VERB', 'DET', 'NOUN', 'PREP', 'PRO', 'NOUN', 'DET', 'NOUN', 'VERB', 'ADV', 'ADJ', 'PART', 'VERB', 'CONJ', 'VERB', 'PREP', 'PRO', 'NOUN', 'PRO', 'VERB', 'DET', 'NOUN', 'ADV', 'VERB', 'DET', 'NOUN', 'CONJ', 'DET', 'ADJ', 'NOUN', 'NOUN', 'ADV', 'VERB', 'ADJ', 'NOUN']


In [ ]:
accuracy_score(english_clean[1], nltk_convert)

0.6491228070175439

**FLAIR**

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('pos-multi')

2020-10-18 15:52:52,658 loading file /root/.flair/models/pos-multi-v0.1.pt


In [ ]:
sentence = Sentence(s)

# predict pos tags
tagger.predict(sentence)

# print sentence with predicted tags
tags_flair = sentence.to_tagged_string()
print(tags_flair)

i <PRON> goed <VERB> to <ADP> the <DET> shop <NOUN> twitter <NOUN> is <AUX> my <PRON> absolutely <ADV> fav <ADJ> social <ADJ> media <NOUN> facebook <NOUN> is <AUX> the <DET> way <NOUN> to <ADP> baaad <VERB> i <PRON> totally <ADV> love <VERB> your <PRON> twts <NOUN> this <PRON> is <AUX> a <DET> subtweet <NOUN> of <ADP> anyone <PRON> in <ADP> particular <ADJ> and <CCONJ> it <PRON> is <AUX> going <VERB> to <PART> annoy <VERB> some <DET> people <NOUN> i <PRON> have <VERB> a <DET> lot <NOUN> of <ADP> phd <NOUN> followers <NOUN> and <CCONJ> friends <NOUN> and <CCONJ> i <PRON> am <AUX> a <DET> stubborn <ADJ> guy <NOUN> myself <PRON> but <CCONJ> one <NUM> thing <NOUN> i <PRON> am <AUX> noticed <VERB> is <VERB> when <ADV> someone <PRON> gets <VERB> a <DET> phd <NOUN> it <PRON> become <VERB> 10 <NUM> harder <ADJ> convince <VERB> them <PRON> they <PRON> are <AUX> wrong <ADJ> i <PRON> have <AUX> actually <ADV> found <VERB> the <DET> opposite <NOUN> in <ADP> my <PRON> discipline <NOUN> the <DET> ph

In [ ]:
flair_convert = []

search = re.findall('\<\w+\>', tags_flair)
for tag in search:
  tag = re.sub('<PROPN>$', 'NOUN', tag)
  tag = re.sub('<AUX>$', 'VERB', tag)
  tag = re.sub('<ADP>$', 'PREP', tag)
  tag = re.sub('<CCONJ>$', 'CONJ', tag)
  tag = re.sub('<PRO>$', 'PRON', tag)

  flair_convert.append(tag.replace('<','').replace('>',''))

print(flair_convert)

['PRON', 'VERB', 'PREP', 'DET', 'NOUN', 'NOUN', 'VERB', 'PRON', 'ADV', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PREP', 'VERB', 'PRON', 'ADV', 'VERB', 'PRON', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'PREP', 'PRON', 'PREP', 'ADJ', 'CONJ', 'PRON', 'VERB', 'VERB', 'PART', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'CONJ', 'PRON', 'VERB', 'DET', 'ADJ', 'NOUN', 'PRON', 'CONJ', 'NUM', 'NOUN', 'PRON', 'VERB', 'VERB', 'VERB', 'ADV', 'PRON', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'NUM', 'ADJ', 'VERB', 'PRON', 'PRON', 'VERB', 'ADJ', 'PRON', 'VERB', 'ADV', 'VERB', 'DET', 'NOUN', 'PREP', 'PRON', 'NOUN', 'DET', 'NOUN', 'VERB', 'ADV', 'ADJ', 'PREP', 'NOUN', 'CONJ', 'NOUN', 'PREP', 'PRON', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN', 'CONJ', 'DET', 'ADJ', 'NOUN', 'NOUN', 'PRON', 'VERB', 'ADJ', 'NOUN']


In [ ]:
accuracy_score(english_clean[1], flair_convert)

0.8070175438596491

**Вывод по парсерам**

Среди русских парсеров самый лучший результат показал PyMorphy (0.9196428571428571), затем идет Наташа (0.875), хуже всех показал себя Mystem (0.8035714285714286)


Среди английских парсеров на первом месте flair (0.8070175438596491), на втором spacy (0.7807017543859649), а хуже всех оказался NLTK (0.6491228070175439)

**ЧАСТЬ 3 - как улучшить код из предыдущего дз?**

Для тестирования программы возьмем рандомный хороший и плохой отзыв с кинопоиска 

In [ ]:
text = 'не красивый \
сильно мудрый \
не сильно мудрый \
Великолепная картина рассказывает об учителе географии, \
который умеет заворожить непростых подростков и показать им предмет с другой стороны. \
Он устраивает интересные экскурсии и даже берет учеников в поход, каждый из них сталкивается с трудностями и опасностями, \
преодоление которых меняет всех участников на всю оставшуюся жизнь \
На фоне работы, наш учитель так же живет с любимой женой, однако та его больше не любит, \
и наш географ отпускает ее к своему лучшему другу потому, что настоящая любовь выше эгоизма, \
настоящая любовь — это стремление и желание того, чтобы твой объект любви был счастлив.\
Одним из испытаний становится и безответная любовь ученицы и соблазн для нашего взрослого географа, \
которому он не поддается, имея твердый и честный характер. Несмотря на пропащность множества окружающих, \
наш географ остается чист \
очень хороший фильм \
История не милого персонажа живущего в несчастливом браке, находящего отдушину только в собственном деле. \
Качественно выбрано место для съемки, красивая, подвижная локация. \
Актерская игра на уровне, молодые актеры показывают опытный профессионализм, \
а игра Константина Хабенского, как всегда безупречна и многогранна. \
Я нежно и трепетно люблю сказки братьев Гримм и их интерпретации. \
Трейлер на эту версию популярной истории выглядел достаточно красиво и интересно, \
и я с нетерпением ждала выхода фильма — тем более, главную роль исполняет девочка, \
которая прекрасно отыграла Беверли в «Оно». И… Чем дальше в лес  \
Первое, что меня смутило — переделка начала истории, где нет отца и безумной мачехи, \
а мать имеет какое-то расстройство. Ладно, можно простить некоторые отступления от оригинальной истории, \
адаптация на то и адаптация. \
Но дальнейшее повествование слишком уныло и однообразно. \
Моменты, которые должны пугать или навевать на какие-то мысли, \
выполнены слишком прямо и топорно, а визуальные эффекты смотрятся достаточно дешево \
и скорее напоминают отечественные выходки, нежели атмосферу хоррора.\
Можно сделать качественную работу, не выходя за пределы низкого рейтинга, но это конечно случай. \
Героиня никакая, к ней не возникает ни сочувствия, ни сопереживания. \
Эмоциональной подоплеки не хватает, чувство долга перед братом высосано \
как таковая близость между родными не показана, не говоря уже о действительно глупых ляпах \
(в лесу не найти ни кореньев, ни ягод, ни грибов, будучи деревенскими жителями, серьезно?). \
Или же почему героиня не разбирается в растениях, если живет рядом с лесом и должна заботиться о себе и семье? \
Фильм похож на черновик фильма, а не хорошую и качественную работу. ' 

In [ ]:
text_list = text.split(' ')
len(text_list)

379

1) Прилагательное + существительное (без не перед прилагательным) - очень часто фильмы описываются такой конструкцией - ужасный фильм / хороший фильм. Чтобы избежать путаницы вида хороший фильм - не хороший фильм отберем только прил + сущ без не перед прил 

2) Наречие + прилагательное - очень хороший / очень плохой, эта конструкция тоже часто используется для описание. Вновь для того, чтобы избежать неоднозначности возьмем только без не перед наречием

3) не + глагол - очень часто используется - типа не понравился / не впечатлил - для плохих отзывов, не разочаровал - для хороших 

4) не + прилагательное - также часто используется. Например, для плохих - не хороший, не впечатляющий; для хороших - не разочаровавший, иногда опечатки типа 'не обыкновенный'

Лучший парсер для русского это пайморфи, поэтому используем его

*выведем прилагательное + существительное и при этом 'не' не стоит перед прилагательным*


*выведем наречие + прилагательное и при этом 'не' не стоит перед наречием*

In [ ]:
tags = []
words = []

adj_noun = ' '
adv_adj = ' '

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
for i in text_list:
  p = morph.parse(i)[0]
  words.append(i)
  tags.append(p.tag.POS)

s = 0  # счетчик
for tag in tags:
  try:
    if tag == 'ADJF' and tags[s + 1] == 'NOUN' and words[s - 1] != 'не':  # прилагательное + существительное без не перед прил
      adj_noun += str(words[s]).replace(',','') + ' ' + str(words[s + 1]).replace(',','') + '; '
      s+=1

    if tag == 'ADVB' and tags[s + 1] == 'ADJF'and words[s - 1] != 'не':   # наречие + прилагательное без не перед наречией
      adv_adj += str(words[s]).replace(',','') + ' ' + str(words[s + 1]).replace(',','') + '; '
      s +=1

    else:
      s +=1
  except:
    IndexError

print('Прилагательное + существительное:', adj_noun)

print('Наречие + прилагательное:', adv_adj)


Прилагательное + существительное:  Великолепная картина; оставшуюся жизнь; находящего отдушину; выбрано место; молодые актеры; а игра; перед братом; 
Наречие + прилагательное:  сильно мудрый; 


*выведем не + глагол и не + прилагательное*

In [ ]:
ne_verb = ' '
ne_adj = ' '

s = 0
for tag in tags:
  try:

    if tag == 'VERB' and words[s - 1] == "не":  # не + глагол 
      ne_verb += str(words[s - 1]) + ' ' + str(words[s]) + '; '
    if tag == 'ADJF' and words[s - 1] == "не":  # не + прилагательное
      ne_adj += str(words[s - 1]) + ' ' + str(words[s]) + '; '
      s +=1
    else:
      s +=1
  except:
    IndexError

print('Не + глагол:', ne_verb)

print('Не + прилагательное:', ne_adj)

Не + глагол:  не возникает; не разбирается; 
Не + прилагательное:  не красивый; не милого; не хорошую; 
